In [28]:
import torch
import torchvision

In [29]:
img_size = 360

LOAD_PATH = '../models/resnet50_dataset_3_lr_0_0001_final.pth'
SAVE_PATH = '../models/final_classifier.pth'

In [30]:
class customThreshold(torch.nn.Module):
    def __init__(self, fake_label, threshold):
        super(customThreshold, self).__init__()
        self.fake_label = fake_label
        self.threshold = threshold

    def forward(self, x):
        prob = torch.sigmoid(x[:, self.fake_label]) # extract probability of fake
        pred = (prob >= self.threshold).float() # threshold probability
        return (prob.view(-1, 1), pred.view(-1, 1))

In [31]:
model = torchvision.models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(LOAD_PATH, map_location=torch.device("cpu")))
model.fc = torch.nn.Sequential(model.fc, customThreshold(1, 0.6))
model = model.eval()

In [32]:
img = torch.randn(1, 3, img_size, img_size)  

with torch.no_grad():
    traced_model = torch.jit.trace(model, img)

torch.jit.save(traced_model, SAVE_PATH)